In [1]:
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision.models import resnet18
from scipy.sparse import load_npz

from tqdm import tqdm
import os
import json
import warnings
warnings.filterwarnings("ignore")

In [2]:
DATA_ROOT = "./data"
batch_size = 128
NUM_WORKERS = 4
SEED = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED)
np.random.seed(SEED)

In [3]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5071, 0.4867, 0.4408],
        std=[0.2675, 0.2565, 0.2761]
    )
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5071, 0.4867, 0.4408],
        std=[0.2675, 0.2565, 0.2761]
    )
])

train_dataset = torchvision.datasets.CIFAR100(
    root="./data", train=True, download=True, transform=transform_train
)

test_dataset = torchvision.datasets.CIFAR100(
    root="./data", train=False, download=True, transform=transform_test
)

train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS
)

test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS
)

100%|██████████| 169M/169M [00:14<00:00, 11.5MB/s] 


### Training

In [4]:
model = resnet18(num_classes=100)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

EPOCHS = 100

for epoch in range(EPOCHS):

    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    model.eval()
    test_loss = 0.0
    test_correct = 0
    test_total = 0

    # Total steps = train + test
    total_steps = len(train_loader) + len(test_loader)

    with tqdm(total=total_steps, 
              desc=f"Epoch [{epoch+1}/{EPOCHS}]", 
              leave=True) as pbar:

        # ================= TRAIN =================
        model.train()
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += targets.size(0)
            train_correct += predicted.eq(targets).sum().item()

            train_acc = 100. * train_correct / train_total

            pbar.set_postfix({
                "train_loss": f"{train_loss/(pbar.n+1):.4f}",
                "train_acc": f"{train_acc:.2f}%",
                "test_loss": f"{test_loss:.4f}",
                "test_acc": f"{0.0:.2f}%"
            })
            pbar.update(1)

        # ================= TEST =================
        model.eval()
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                test_total += targets.size(0)
                test_correct += predicted.eq(targets).sum().item()

                test_acc = 100. * test_correct / test_total

                pbar.set_postfix({
                    "train_loss": f"{train_loss/len(train_loader):.4f}",
                    "train_acc": f"{train_acc:.2f}%",
                    "test_loss": f"{test_loss/(test_total/inputs.size(0)):.4f}",
                    "test_acc": f"{test_acc:.2f}%"
                })
                pbar.update(1)


Epoch [100/100]: 100%|██████████| 470/470 [00:27<00:00, 17.25it/s, train_loss=0.9602, train_acc=71.72%, test_loss=0.2019, test_acc=58.46%]


In [5]:
torch.save(
    {
        "epoch": 100,
        "model_state_dict": model.state_dict(),
    },
    "resnet18_cifar100_trained_model.pth"
)

### Class Level Unlearning

In [11]:
def evaluate_accuracy(model,test_loader,device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    acc = correct / total
    return round(acc,3)

In [6]:
def split_dataset_by_class(dataset, forget_class):
    retain_indices = []
    forget_indices = []

    for idx in range(len(dataset)):
        _, label = dataset[idx]
        if label == forget_class:
            forget_indices.append(idx)
        else:
            retain_indices.append(idx)

    retain_subset = Subset(dataset, retain_indices)
    forget_subset = Subset(dataset, forget_indices)
    
    return retain_subset, forget_subset

forget_class = 1

# Use this for both train and test datasets
retain_train_ds, forget_train_ds = split_dataset_by_class(train_dataset, forget_class)
retain_test_ds, forget_test_ds = split_dataset_by_class(test_dataset, forget_class)

# Wrap them in DataLoaders
retain_train_dl = DataLoader(retain_train_ds, batch_size=train_loader.batch_size, shuffle=True, num_workers=2)
forget_train_dl = DataLoader(forget_train_ds, batch_size=train_loader.batch_size, shuffle=True, num_workers=2)

retain_test_dl = DataLoader(retain_test_ds, batch_size=test_loader.batch_size, shuffle=False, num_workers=2)
forget_test_dl = DataLoader(forget_test_ds, batch_size=test_loader.batch_size, shuffle=False, num_workers=2)

In [12]:
print("Evaluating model accuracy:")

acc_retain_train = evaluate_accuracy(model, retain_train_dl, device)
print(f"Retain Train Accuracy: {acc_retain_train:.2%}")

acc_forget_train = evaluate_accuracy(model, forget_train_dl, device)
print(f"Forget Train Accuracy: {acc_forget_train:.2%}")

acc_retain_test = evaluate_accuracy(model, retain_test_dl, device)
print(f"Retain Test Accuracy: {acc_retain_test:.2%}")

acc_forget_test = evaluate_accuracy(model, forget_test_dl, device)
print(f"Forget Test Accuracy: {acc_forget_test:.2%}")

Evaluating model accuracy:
Retain Train Accuracy: 67.20%
Forget Train Accuracy: 84.00%
Retain Test Accuracy: 58.20%
Forget Test Accuracy: 82.00%


#### Random Labelling

**Version 1 (aggresive Random Labelling): No use of retain data, for the same forget sample each time its a different random label**

In [19]:
model = resnet18(num_classes=100)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()
model = model.to(device)

checkpoint_path = "/kaggle/working/resnet18_cifar100_trained_model.pth"
model.load_state_dict(torch.load(checkpoint_path, map_location=device)['model_state_dict'])

num_classes = 100
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
epoch = 1

while True:
    model.train()
    for inputs, targets in forget_train_dl:
        inputs, targets = inputs.to(device), targets.to(device)
        random_targets = torch.randint(0, num_classes, targets.shape, device=device)
        same_mask = random_targets == targets
        while same_mask.any():
            random_targets[same_mask] = torch.randint(
                0, num_classes, (same_mask.sum(),), device=device
            )
            same_mask = random_targets == targets
            
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, random_targets)
        loss.backward()
        optimizer.step()

    acc_forget_train = evaluate_accuracy(model, forget_train_dl, device)
    acc_retain_train = evaluate_accuracy(model, retain_train_dl, device)

    print(f"Epoch: {epoch} | Retain Train Accuracy: {acc_retain_train} | Forget Train Accuracy: {acc_forget_train}")
    epoch+=1
    if acc_forget_train<=0.02:
        print('Convergence Reached')
        break

Epoch: 1 | Retain Train Accuracy: 0.696 | Forget Train Accuracy: 0.472
Epoch: 2 | Retain Train Accuracy: 0.687 | Forget Train Accuracy: 0.26
Epoch: 3 | Retain Train Accuracy: 0.673 | Forget Train Accuracy: 0.144
Epoch: 4 | Retain Train Accuracy: 0.661 | Forget Train Accuracy: 0.066
Epoch: 5 | Retain Train Accuracy: 0.653 | Forget Train Accuracy: 0.052
Epoch: 6 | Retain Train Accuracy: 0.643 | Forget Train Accuracy: 0.04
Epoch: 7 | Retain Train Accuracy: 0.639 | Forget Train Accuracy: 0.036
Epoch: 8 | Retain Train Accuracy: 0.636 | Forget Train Accuracy: 0.024
Epoch: 9 | Retain Train Accuracy: 0.634 | Forget Train Accuracy: 0.018
Convergence Reached


In [20]:
print("Evaluating model accuracy:")

acc_retain_train = evaluate_accuracy(model, retain_train_dl, device)
print(f"Retain Train Accuracy: {acc_retain_train:.2%}")

acc_forget_train = evaluate_accuracy(model, forget_train_dl, device)
print(f"Forget Train Accuracy: {acc_forget_train:.2%}")

acc_retain_test = evaluate_accuracy(model, retain_test_dl, device)
print(f"Retain Test Accuracy: {acc_retain_test:.2%}")

acc_forget_test = evaluate_accuracy(model, forget_test_dl, device)
print(f"Forget Test Accuracy: {acc_forget_test:.2%}")

Evaluating model accuracy:
Retain Train Accuracy: 63.40%
Forget Train Accuracy: 2.40%
Retain Test Accuracy: 55.20%
Forget Test Accuracy: 1.00%


**Version 2 (Fixed Random Labelling): No use of retain data, we replce the label with random label for forget set then proceed for training**

In [21]:
from torch.utils.data import Dataset

class FixedRandomLabelDataset(Dataset):
    def __init__(self, base_dataset, num_classes, seed=42):
        self.base_dataset = base_dataset
        self.num_classes = num_classes

        # Set seed for reproducibility
        g = torch.Generator()
        g.manual_seed(seed)

        self.fixed_labels = []

        for i in range(len(base_dataset)):
            _, original_label = base_dataset[i]

            # Sample from [0, num_classes-2]
            rand_label = torch.randint(
                0, num_classes - 1, (1,), generator=g
            ).item()

            # Shift if needed to avoid original label
            if rand_label >= original_label:
                rand_label += 1

            self.fixed_labels.append(rand_label)

    def __len__(self):
        return len(self.base_dataset)

    def __getitem__(self, index):
        x, _ = self.base_dataset[index]
        return x, self.fixed_labels[index]

fixed_forget_dataset = FixedRandomLabelDataset(forget_train_ds, num_classes)
fixed_forget_train_dl = torch.utils.data.DataLoader(fixed_forget_dataset, batch_size=128, shuffle=True)

In [22]:
model = resnet18(num_classes=100)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()
model = model.to(device)

checkpoint_path = "/kaggle/working/resnet18_cifar100_trained_model.pth"
model.load_state_dict(torch.load(checkpoint_path, map_location=device)['model_state_dict'])

num_classes = 100
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
epoch = 1

while True:
    model.train()
    for inputs, targets in fixed_forget_train_dl:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    acc_forget_train = evaluate_accuracy(model, forget_train_dl, device)
    acc_retain_train = evaluate_accuracy(model, retain_train_dl, device)

    print(f"Epoch: {epoch} | Retain Train Accuracy: {acc_retain_train} | Forget Train Accuracy: {acc_forget_train}")
    epoch+=1
    if acc_forget_train<=0.02:
        print('Convergence Reached')
        break

Epoch: 1 | Retain Train Accuracy: 0.692 | Forget Train Accuracy: 0.508
Epoch: 2 | Retain Train Accuracy: 0.684 | Forget Train Accuracy: 0.244
Epoch: 3 | Retain Train Accuracy: 0.67 | Forget Train Accuracy: 0.108
Epoch: 4 | Retain Train Accuracy: 0.658 | Forget Train Accuracy: 0.062
Epoch: 5 | Retain Train Accuracy: 0.648 | Forget Train Accuracy: 0.052
Epoch: 6 | Retain Train Accuracy: 0.641 | Forget Train Accuracy: 0.038
Epoch: 7 | Retain Train Accuracy: 0.639 | Forget Train Accuracy: 0.026
Epoch: 8 | Retain Train Accuracy: 0.636 | Forget Train Accuracy: 0.024
Epoch: 9 | Retain Train Accuracy: 0.632 | Forget Train Accuracy: 0.024
Epoch: 10 | Retain Train Accuracy: 0.632 | Forget Train Accuracy: 0.022
Epoch: 11 | Retain Train Accuracy: 0.63 | Forget Train Accuracy: 0.03
Epoch: 12 | Retain Train Accuracy: 0.63 | Forget Train Accuracy: 0.022
Epoch: 13 | Retain Train Accuracy: 0.632 | Forget Train Accuracy: 0.018
Convergence Reached


In [23]:
print("Evaluating model accuracy:")

acc_retain_train = evaluate_accuracy(model, retain_train_dl, device)
print(f"Retain Train Accuracy: {acc_retain_train:.2%}")

acc_forget_train = evaluate_accuracy(model, forget_train_dl, device)
print(f"Forget Train Accuracy: {acc_forget_train:.2%}")

acc_retain_test = evaluate_accuracy(model, retain_test_dl, device)
print(f"Retain Test Accuracy: {acc_retain_test:.2%}")

acc_forget_test = evaluate_accuracy(model, forget_test_dl, device)
print(f"Forget Test Accuracy: {acc_forget_test:.2%}")

Evaluating model accuracy:
Retain Train Accuracy: 63.30%
Forget Train Accuracy: 2.40%
Retain Test Accuracy: 55.00%
Forget Test Accuracy: 1.00%


### Similar Label Strategy [As mentioned in Robust Machine Unlearning for Quantized Networks by Yujia Tong et al ICCV 2025]

It selects the closest wrong label, for example image x has label yi we feed the model get its predicted probability choose the closest class that has probability value closer to yi

In [27]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset

model = resnet18(num_classes=100)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()
model = model.to(device)

checkpoint_path = "/kaggle/working/resnet18_cifar100_trained_model.pth"
model.load_state_dict(torch.load(checkpoint_path, map_location=device)['model_state_dict'])


class SimilarLabelDataset(Dataset):
    def __init__(self, base_dataset, model, device):
        self.base_dataset = base_dataset
        self.model = model
        self.device = device

        self.model.eval()
        self.new_labels = []

        with torch.no_grad():
            for i in range(len(base_dataset)):
                x, y_true = base_dataset[i]
                x = x.unsqueeze(0).to(device)

                logits = model(x)
                probs = F.softmax(logits, dim=1).squeeze(0)

                p_true = probs[y_true]

                # Compute distance to true class probability
                distances = torch.abs(probs - p_true)

                # Exclude original class
                distances[y_true] = float("inf")

                # Choose class with minimum distance
                new_label = torch.argmin(distances).item()

                self.new_labels.append(new_label)

    def __len__(self):
        return len(self.base_dataset)

    def __getitem__(self, index):
        x, _ = self.base_dataset[index]
        return x, self.new_labels[index]


similarLabel_forget_dataset = SimilarLabelDataset(forget_train_ds, model, device)
similarLabel_forget_train_dl = torch.utils.data.DataLoader(similarLabel_forget_dataset, batch_size=128, shuffle=True)

In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
epoch = 1

while True:
    model.train()
    for inputs, targets in similarLabel_forget_train_dl:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    acc_forget_train = evaluate_accuracy(model, forget_train_dl, device)
    acc_retain_train = evaluate_accuracy(model, retain_train_dl, device)

    print(f"Epoch: {epoch} | Retain Train Accuracy: {acc_retain_train} | Forget Train Accuracy: {acc_forget_train}")
    epoch+=1
    if acc_forget_train<=0.02:
        print('Convergence Reached')
        break

Epoch: 1 | Retain Train Accuracy: 0.694 | Forget Train Accuracy: 0.514
Epoch: 2 | Retain Train Accuracy: 0.685 | Forget Train Accuracy: 0.248
Epoch: 3 | Retain Train Accuracy: 0.681 | Forget Train Accuracy: 0.124
Epoch: 4 | Retain Train Accuracy: 0.675 | Forget Train Accuracy: 0.068
Epoch: 5 | Retain Train Accuracy: 0.669 | Forget Train Accuracy: 0.048
Epoch: 6 | Retain Train Accuracy: 0.665 | Forget Train Accuracy: 0.032
Epoch: 7 | Retain Train Accuracy: 0.668 | Forget Train Accuracy: 0.022
Epoch: 8 | Retain Train Accuracy: 0.665 | Forget Train Accuracy: 0.016
Convergence Reached


In [29]:
print("Evaluating model accuracy:")

acc_retain_train = evaluate_accuracy(model, retain_train_dl, device)
print(f"Retain Train Accuracy: {acc_retain_train:.2%}")

acc_forget_train = evaluate_accuracy(model, forget_train_dl, device)
print(f"Forget Train Accuracy: {acc_forget_train:.2%}")

acc_retain_test = evaluate_accuracy(model, retain_test_dl, device)
print(f"Retain Test Accuracy: {acc_retain_test:.2%}")

acc_forget_test = evaluate_accuracy(model, forget_test_dl, device)
print(f"Forget Test Accuracy: {acc_forget_test:.2%}")

Evaluating model accuracy:
Retain Train Accuracy: 66.50%
Forget Train Accuracy: 2.20%
Retain Test Accuracy: 57.70%
Forget Test Accuracy: 0.00%


#### Observation:
Replacing the true label with a "semantically close" wrong label (closest in predicted probability space) injects much less disruptive noise into the gradients than fully random wrong labels as suggested in the paper as opposed to aggresive random labelling or fixed random labelling.

### Sample Unlearning

In [30]:
def split_dataset_random(dataset, forget_fraction=0.2, seed=42):
    np.random.seed(seed)

    num_samples = len(dataset)
    indices = np.arange(num_samples)
    np.random.shuffle(indices)

    forget_size = int(forget_fraction * num_samples)

    forget_indices = indices[:forget_size]
    retain_indices = indices[forget_size:]

    retain_subset = Subset(dataset, retain_indices)
    forget_subset = Subset(dataset, forget_indices)

    return retain_subset, forget_subset

forget_fraction = 0.1  # 10% randomly selected for forget

retain_train_ds, forget_train_ds = split_dataset_random(
    train_dataset,
    forget_fraction=forget_fraction,
    seed=42
)

retain_train_dl = DataLoader(
    retain_train_ds,
    batch_size=train_loader.batch_size,
    shuffle=True,
    num_workers=2
)

forget_train_dl = DataLoader(
    forget_train_ds,
    batch_size=train_loader.batch_size,
    shuffle=True,
    num_workers=2
)

In [32]:
model = resnet18(num_classes=100)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()
model = model.to(device)

checkpoint_path = "/kaggle/working/resnet18_cifar100_trained_model.pth"
model.load_state_dict(torch.load(checkpoint_path, map_location=device)['model_state_dict'])


print("Evaluating model accuracy:")

acc_retain_train = evaluate_accuracy(model, retain_train_dl, device)
print(f"Retain Train Accuracy: {acc_retain_train:.2%}")

acc_forget_train = evaluate_accuracy(model, forget_train_dl, device)
print(f"Forget Train Accuracy: {acc_forget_train:.2%}")

acc_test = evaluate_accuracy(model, test_loader, device)
print(f"Test Accuracy: {acc_test:.2%}")

Evaluating model accuracy:
Retain Train Accuracy: 67.60%
Forget Train Accuracy: 67.70%
Test Accuracy: 58.50%


#### Version 1: Aggresive Random Labelling

In [35]:
num_classes = 100
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
model.load_state_dict(torch.load(checkpoint_path, map_location=device)['model_state_dict'])

for epoch in range(1,6):
    model.train()
    for inputs, targets in forget_train_dl:
        inputs, targets = inputs.to(device), targets.to(device)
        random_targets = torch.randint(0, num_classes, targets.shape, device=device)
        same_mask = random_targets == targets
        while same_mask.any():
            random_targets[same_mask] = torch.randint(
                0, num_classes, (same_mask.sum(),), device=device
            )
            same_mask = random_targets == targets
            
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, random_targets)
        loss.backward()
        optimizer.step()

    acc_forget_train = evaluate_accuracy(model, forget_train_dl, device)
    acc_retain_train = evaluate_accuracy(model, retain_train_dl, device)

    print(f"Epoch: {epoch} | Retain Train Accuracy: {acc_retain_train} | Forget Train Accuracy: {acc_forget_train}")

Epoch: 1 | Retain Train Accuracy: 0.697 | Forget Train Accuracy: 0.699
Epoch: 2 | Retain Train Accuracy: 0.531 | Forget Train Accuracy: 0.508
Epoch: 3 | Retain Train Accuracy: 0.344 | Forget Train Accuracy: 0.318
Epoch: 4 | Retain Train Accuracy: 0.228 | Forget Train Accuracy: 0.208
Epoch: 5 | Retain Train Accuracy: 0.161 | Forget Train Accuracy: 0.147


In [36]:
print("Evaluating model accuracy:")

acc_retain_train = evaluate_accuracy(model, retain_train_dl, device)
print(f"Retain Train Accuracy: {acc_retain_train:.2%}")

acc_forget_train = evaluate_accuracy(model, forget_train_dl, device)
print(f"Forget Train Accuracy: {acc_forget_train:.2%}")

acc_test = evaluate_accuracy(model, test_loader, device)
print(f"Test Accuracy: {acc_test:.2%}")

Evaluating model accuracy:
Retain Train Accuracy: 16.00%
Forget Train Accuracy: 15.50%
Test Accuracy: 14.00%


#### Version 2: Fixed Random Labelling

In [37]:
fixed_forget_dataset = FixedRandomLabelDataset(forget_train_ds, num_classes)
fixed_forget_train_dl = torch.utils.data.DataLoader(fixed_forget_dataset, batch_size=128, shuffle=True)

In [38]:
num_classes = 100
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
model.load_state_dict(torch.load(checkpoint_path, map_location=device)['model_state_dict'])

for epoch in range(1,6):
    model.train()
    for inputs, targets in fixed_forget_train_dl:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    acc_forget_train = evaluate_accuracy(model, forget_train_dl, device)
    acc_retain_train = evaluate_accuracy(model, retain_train_dl, device)

    print(f"Epoch: {epoch} | Retain Train Accuracy: {acc_retain_train} | Forget Train Accuracy: {acc_forget_train}")

Epoch: 1 | Retain Train Accuracy: 0.697 | Forget Train Accuracy: 0.697
Epoch: 2 | Retain Train Accuracy: 0.52 | Forget Train Accuracy: 0.508
Epoch: 3 | Retain Train Accuracy: 0.338 | Forget Train Accuracy: 0.313
Epoch: 4 | Retain Train Accuracy: 0.215 | Forget Train Accuracy: 0.204
Epoch: 5 | Retain Train Accuracy: 0.158 | Forget Train Accuracy: 0.156


In [39]:
print("Evaluating model accuracy:")

acc_retain_train = evaluate_accuracy(model, retain_train_dl, device)
print(f"Retain Train Accuracy: {acc_retain_train:.2%}")

acc_forget_train = evaluate_accuracy(model, forget_train_dl, device)
print(f"Forget Train Accuracy: {acc_forget_train:.2%}")

acc_test = evaluate_accuracy(model, test_loader, device)
print(f"Test Accuracy: {acc_test:.2%}")

Evaluating model accuracy:
Retain Train Accuracy: 16.00%
Forget Train Accuracy: 15.40%
Test Accuracy: 13.80%


#### Version 3: Similar Label

In [40]:
checkpoint_path = "/kaggle/working/resnet18_cifar100_trained_model.pth"
model.load_state_dict(torch.load(checkpoint_path, map_location=device)['model_state_dict'])

similarLabel_forget_dataset = SimilarLabelDataset(forget_train_ds, model, device)
similarLabel_forget_train_dl = torch.utils.data.DataLoader(similarLabel_forget_dataset, batch_size=128, shuffle=True)

In [42]:
num_classes = 100
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
model.load_state_dict(torch.load(checkpoint_path, map_location=device)['model_state_dict'])

for epoch in range(1,6):
    model.train()
    for inputs, targets in similarLabel_forget_train_dl:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    acc_forget_train = evaluate_accuracy(model, forget_train_dl, device)
    acc_retain_train = evaluate_accuracy(model, retain_train_dl, device)

    print(f"Epoch: {epoch} | Retain Train Accuracy: {acc_retain_train} | Forget Train Accuracy: {acc_forget_train}")

Epoch: 1 | Retain Train Accuracy: 0.717 | Forget Train Accuracy: 0.709
Epoch: 2 | Retain Train Accuracy: 0.657 | Forget Train Accuracy: 0.646
Epoch: 3 | Retain Train Accuracy: 0.6 | Forget Train Accuracy: 0.592
Epoch: 4 | Retain Train Accuracy: 0.568 | Forget Train Accuracy: 0.548
Epoch: 5 | Retain Train Accuracy: 0.543 | Forget Train Accuracy: 0.524


In [43]:
print("Evaluating model accuracy:")

acc_retain_train = evaluate_accuracy(model, retain_train_dl, device)
print(f"Retain Train Accuracy: {acc_retain_train:.2%}")

acc_forget_train = evaluate_accuracy(model, forget_train_dl, device)
print(f"Forget Train Accuracy: {acc_forget_train:.2%}")

acc_test = evaluate_accuracy(model, test_loader, device)
print(f"Test Accuracy: {acc_test:.2%}")

Evaluating model accuracy:
Retain Train Accuracy: 54.10%
Forget Train Accuracy: 51.50%
Test Accuracy: 46.20%


#### Version 4: Fixed Random Labelling with retain data

In [49]:
fixed_forget_dataset[0][1],forget_train_ds[0][1]

(6, 63)

In [45]:
from torch.utils.data import ConcatDataset

fixed_forget_retain_train_dataset = ConcatDataset([
    retain_train_ds,          # original labels
    fixed_forget_dataset      # modified labels
])

fixed_forget_retain_train_dl = torch.utils.data.DataLoader(
    fixed_forget_retain_train_dataset,
    batch_size=128,
    shuffle=True,
    num_workers=2
)

In [46]:
num_classes = 100
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
model.load_state_dict(torch.load(checkpoint_path, map_location=device)['model_state_dict'])

for epoch in range(1,6):
    model.train()
    for inputs, targets in fixed_forget_retain_train_dl:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    acc_forget_train = evaluate_accuracy(model, forget_train_dl, device)
    acc_retain_train = evaluate_accuracy(model, retain_train_dl, device)

    print(f"Epoch: {epoch} | Retain Train Accuracy: {acc_retain_train} | Forget Train Accuracy: {acc_forget_train}")

Epoch: 1 | Retain Train Accuracy: 0.823 | Forget Train Accuracy: 0.825
Epoch: 2 | Retain Train Accuracy: 0.843 | Forget Train Accuracy: 0.843
Epoch: 3 | Retain Train Accuracy: 0.855 | Forget Train Accuracy: 0.846
Epoch: 4 | Retain Train Accuracy: 0.864 | Forget Train Accuracy: 0.854
Epoch: 5 | Retain Train Accuracy: 0.872 | Forget Train Accuracy: 0.856


In [50]:
print("Evaluating model accuracy:")

acc_retain_train = evaluate_accuracy(model, retain_train_dl, device)
print(f"Retain Train Accuracy: {acc_retain_train:.2%}")

acc_forget_train = evaluate_accuracy(model, forget_train_dl, device)
print(f"Forget Train Accuracy: {acc_forget_train:.2%}")

acc_test = evaluate_accuracy(model, test_loader, device)
print(f"Test Accuracy: {acc_test:.2%}")

Evaluating model accuracy:
Retain Train Accuracy: 87.00%
Forget Train Accuracy: 86.10%
Test Accuracy: 72.90%


#### Version 5: Similar Label with retain mixing

In [51]:
similarLabel_forget_retain_train_dataset = ConcatDataset([
    retain_train_ds,          # original labels
    similarLabel_forget_dataset      # modified labels
])

similarLabel_forget_retain_train_dl = torch.utils.data.DataLoader(
    similarLabel_forget_retain_train_dataset,
    batch_size=128,
    shuffle=True,
    num_workers=2
)

In [52]:
num_classes = 100
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
model.load_state_dict(torch.load(checkpoint_path, map_location=device)['model_state_dict'])

for epoch in range(1,6):
    model.train()
    for inputs, targets in similarLabel_forget_retain_train_dl:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    acc_forget_train = evaluate_accuracy(model, forget_train_dl, device)
    acc_retain_train = evaluate_accuracy(model, retain_train_dl, device)

    print(f"Epoch: {epoch} | Retain Train Accuracy: {acc_retain_train} | Forget Train Accuracy: {acc_forget_train}")

Epoch: 1 | Retain Train Accuracy: 0.826 | Forget Train Accuracy: 0.826
Epoch: 2 | Retain Train Accuracy: 0.844 | Forget Train Accuracy: 0.845
Epoch: 3 | Retain Train Accuracy: 0.853 | Forget Train Accuracy: 0.848
Epoch: 4 | Retain Train Accuracy: 0.859 | Forget Train Accuracy: 0.852
Epoch: 5 | Retain Train Accuracy: 0.87 | Forget Train Accuracy: 0.854


In [53]:
print("Evaluating model accuracy:")

acc_retain_train = evaluate_accuracy(model, retain_train_dl, device)
print(f"Retain Train Accuracy: {acc_retain_train:.2%}")

acc_forget_train = evaluate_accuracy(model, forget_train_dl, device)
print(f"Forget Train Accuracy: {acc_forget_train:.2%}")

acc_test = evaluate_accuracy(model, test_loader, device)
print(f"Test Accuracy: {acc_test:.2%}")

Evaluating model accuracy:
Retain Train Accuracy: 86.80%
Forget Train Accuracy: 85.90%
Test Accuracy: 72.90%
